In [8]:
import pandas as pd

# 데이터셋 불러오기
file_path = "university_cafe_reviews.csv"  # 파일 경로 지정
df = pd.read_csv(file_path)

# 데이터셋 확인
print(f"데이터셋 크기: {df.shape}")
print(df.head())

데이터셋 크기: (115817, 8)
    대학교 이름  가게 이름 가게 평균 평점 리뷰어 리뷰수  \
0  서울시립대학교  컴투레스트     4.55     326   
1  서울시립대학교  컴투레스트     4.55     514   
2  서울시립대학교  컴투레스트     4.55     379   
3  서울시립대학교  컴투레스트     4.55      97   
4  서울시립대학교  컴투레스트     4.55     324   

                                              리뷰 텍스트    리뷰 날짜  가게 방문수  \
0  서울권에 있는 카페 거의 다가봤는데 개인적으로 1등 1티어입니다 친구들 회기오면 무...  11.11.월       7   
1  분위기도 좋은데 커피가 너무 맛있어요. 디카페인커피는 경험상 일반 아메리카노보다 맛...  10.26.토       1   
2  오랫만에 왔는데 여전히 커피도 맛있네요!! 시나몬카푸치노였는데 은은한 시나몬향과 살...  10.22.화       1   
3  한번 와보고 싶었던 곳인데 드디어 와봅니다.\n판나코타 궁금했는데 이건 1인 1개 ...   7.25.목       1   
4  오늘 1일.2카페!\n동생이 경희대 근처 커피집 다 이쁘고 맛나다고 ㅋ\n지나가다 ...    7.4.목       1   

                                                 태그들  
0  커피가 맛있어요, 디저트가 맛있어요, 특별한 메뉴가 있어요, 아늑해요, 차분한 분위기에요  
1                                커피가 맛있어요, 디저트가 맛있어요  
2                                 커피가 맛있어요, 대화하기 좋아요  
3            디저트가 맛있어요, 특별한 메뉴가 있어요, 인테리어가 멋져요, 아늑해요  
4  커피가 맛있어요, 인테리어가 멋져요, 사진이 잘 나와요, 대

In [1]:
from konlpy.tag import Okt
import pandas as pd

# Okt 형태소 분석기 초기화
okt = Okt()

# 1. 모든 리뷰 데이터에 대해 형태소 분류 및 새로운 피처 생성
def preprocess_reviews(df):
    def process_text(text):
        if not isinstance(text, str):
            return ""
        tokens = okt.pos(text, stem=True)  # 형태소 분석 및 원형 복원
        filtered_tokens = [word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective', 'Adverb']]
        return " ".join(filtered_tokens)
    
    # 리뷰 텍스트 컬럼을 형태소 분류하여 새로운 피처 생성
    df['형태소_분류_리뷰'] = df['리뷰 텍스트'].apply(process_text)
    return df

# 2. 대학교 이름과 키워드로 가게 검색 및 형태소 리뷰 합치기
def search_and_aggregate(university_name, keyword, df):
    # 2.1 대학교 이름으로 필터링
    university_filtered = df[df['대학교 이름'] == university_name]
    if university_filtered.empty:
        return None, f"'{university_name}'에 해당하는 데이터가 없습니다."

    # 2.2 키워드를 포함하는 가게 이름 필터링
    filtered_gazebo_names = university_filtered[
        university_filtered['형태소_분류_리뷰'].str.contains(keyword)
    ]['가게 이름'].unique()

    if len(filtered_gazebo_names) == 0:
        return None, f"'{university_name}'에서 키워드 '{keyword}'를 포함하는 데이터가 없습니다."

    # 2.3 필터링된 가게의 모든 리뷰 합치기
    final_filtered = university_filtered[university_filtered['가게 이름'].isin(filtered_gazebo_names)]
    grouped_reviews = final_filtered.groupby('가게 이름')['형태소_분류_리뷰'].apply(lambda x: " ".join(x)).reset_index()
    grouped_reviews.columns = ['가게 이름', '리뷰 총합본']
    grouped_reviews['대학교 이름'] = university_name
    grouped_reviews['검색 키워드'] = keyword

    # 열 순서 정리
    grouped_reviews = grouped_reviews[['대학교 이름', '검색 키워드', '가게 이름', '리뷰 총합본']]
    return grouped_reviews, None

# 3. 실행
# 데이터 불러오기
df = pd.read_csv("university_cafe_reviews.csv")  # 예: 대학별 카페 리뷰 데이터셋
df['리뷰 텍스트'] = df['리뷰 텍스트'].fillna("")

# 형태소 분류 및 새로운 피처 생성
df = preprocess_reviews(df)

# 사용자 입력
print("===== 리뷰 필터링 및 전처리 시스템 =====")
university_name = input("분석할 대학교 이름을 입력하세요: ").strip()
keyword = input("리뷰에서 검색할 키워드를 입력하세요: ").strip()

# 검색 및 결과 출력
result, error_message = search_and_aggregate(university_name, keyword, df)

if error_message:
    print(error_message)
else:
    print(result)

    # 결과 미리보기
    print("\n검색 결과 미리보기:")
    print(result.head(10))

    # 전체 결과를 CSV로 저장할지 여부 묻기
    save_option = input("\n결과를 CSV로 저장하시겠습니까? (y/n): ").strip().lower()
    if save_option == 'y':
        file_name = f"{university_name}_{keyword}_filtered_reviews.csv"
        result.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"결과가 '{file_name}'에 저장되었습니다.")
    else:
        print("결과가 저장되지 않았습니다.")


===== 리뷰 필터링 및 전처리 시스템 =====


분석할 대학교 이름을 입력하세요:  명지대학교 인문캠퍼스
리뷰에서 검색할 키워드를 입력하세요:  딸기


         대학교 이름 검색 키워드          가게 이름  \
0   명지대학교 인문캠퍼스     딸기           GRAP   
1   명지대학교 인문캠퍼스     딸기         글룩베이커리   
2   명지대학교 인문캠퍼스     딸기    대단한탕후루 명지대점   
3   명지대학교 인문캠퍼스     딸기     데니스커피 명지대점   
4   명지대학교 인문캠퍼스     딸기     디저트39 명지대점   
5   명지대학교 인문캠퍼스     딸기          매치걸빈스   
6   명지대학교 인문캠퍼스     딸기   메가MGC커피 명지대점   
7   명지대학교 인문캠퍼스     딸기          무지개카페   
8   명지대학교 인문캠퍼스     딸기         발루토 핀치   
9   명지대학교 인문캠퍼스     딸기        설빙 명지대점   
10  명지대학교 인문캠퍼스     딸기     셀렉토커피 남가좌점   
11  명지대학교 인문캠퍼스     딸기         스케네브라운   
12  명지대학교 인문캠퍼스     딸기       시냇가에심은나무   
13  명지대학교 인문캠퍼스     딸기            쏘네당   
14  명지대학교 인문캠퍼스     딸기         오월의 이유   
15  명지대학교 인문캠퍼스     딸기     카페인중독 명지대점   
16  명지대학교 인문캠퍼스     딸기  컴포즈커피 명지대사거리점   
17  명지대학교 인문캠퍼스     딸기   컴포즈커피 명지대정문점   
18  명지대학교 인문캠퍼스     딸기          크레페허브   
19  명지대학교 인문캠퍼스     딸기     팔공티 명지전문대점   
20  명지대학교 인문캠퍼스     딸기             폴센   
21  명지대학교 인문캠퍼스     딸기        프에프에케이크   
22  명지대학교 인문캠퍼스     딸기      하이롱 디저트가게   
23  명지대학교 인문캠퍼스 


결과를 CSV로 저장하시겠습니까? (y/n):  ㅜ


결과가 저장되지 않았습니다.
